In [4]:
import os
import sys
import numpy as np

# local imports
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
import data_loader
#import plotting

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../experiments')))
import utils

2025-03-31 14:24:34.893112: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-31 14:24:34.907443: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743423874.923910  288991 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743423874.928923  288991 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-31 14:24:34.946912: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [5]:
emb_dir = '../../../artifacts/embeddings/'

# huggingface model:
model = 'bert-base-uncased'
pooling = 'mean'
batch_size = 8

# openai model:
#model = 'text-embedding-3-small'
#pooling = ''

### Example 1: Big dataset with train-dev-test split

In [10]:
dataset_name = 'jigsaw'
local_dir = '../../../data/jigsaw_bias'

dataset = utils.get_dataset_with_embeddings(emb_dir, dataset_name, model, pooling, batch_size, local_dir)

X_train, emb_train, y_train, g_train, class_weights, group_weights = dataset.get_split('train')
X_dev, emb_dev, y_dev, g_dev, _, _ = dataset.get_split('dev')
X_test, emb_test, y_test, g_test, _, _ = dataset.get_split('test')

# X_* texts
# emb_* embeddings
# y_* class labels
# g_* protected group labels (one-hot encoded)

print("classes: ", dataset.class_names)
print("protected groups: ", dataset.group_names)

load JigsawBias with option: single-class


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Using Bert with CUDA/GPU
no optmizer specified, default to AdamW
embed train split...
load precomputed embeddings for train set
embed dev split...
load precomputed embeddings for dev set
embed test split...
load precomputed embeddings for test set
classes:  ['not toxic', 'toxic']
protected groups:  ['female', 'male', 'transgender', 'white', 'asian', 'black', 'latino', 'atheist', 'buddhist', 'christian', 'hindu', 'jewish', 'muslim', 'heterosexual', 'bisexual', 'homosexual_gay_or_lesbian', 'intellectual_or_learning_disability', 'physical_disability', 'psychiatric_or_mental_illness']


### Example 2: Small dataset with cross-validation

In [14]:
dataset_name = 'crows_pairs'
local_dir = None

dataset = utils.get_dataset_with_embeddings(emb_dir, dataset_name, model, pooling, batch_size, local_dir)
print("available splits: ", dataset.splits)

for fold_id in range(dataset.n_folds):
    data_dict = dataset.get_cv_split(fold_id)
    X_train, emb_train, y_train, g_train, cw, gw = data_dict['train']
    X_test, emb_test, y_test, g_test, _, _ = data_dict['test']

    # X_* texts
    # emb_* embeddings
    # y_* class labels
    # g_* protected group labels (one-hot encoded)

print("classes: ", dataset.class_names)
print("protected groups: ", dataset.group_names)

load crowspairs


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Using Bert with CUDA/GPU
no optmizer specified, default to AdamW
no loss function specified, default to cross entropy
embed test split...
load precomputed embeddings for test set
available splits:  ['test']
classes:  ['stereo', 'antistereo']
protected groups:  ['black', 'white', 'asian', 'poor', 'rich', 'male', 'female', 'physical disability, sickness', 'mental_disability_illness', 'foreigner, immigrant, undocumented', 'canadian', 'us american', 'mexican', 'chinese', 'italian', 'british', 'russian', 'indian', 'homosexual', 'heterosexual', 'overweight', 'underweight', 'fit', 'tall', 'short', 'christian', 'muslim', 'jewish', 'old', 'young']
